In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
                            .appName("SparkStreamingWordCount").getOrCreate()



In [3]:
# ensure running below command in linux terminal
# nc -lk 9999
# spark streaming works as dataframe/sql
linesDf = spark\
        .readStream\
        .format("socket")\
        .option("host", "192.168.93.128")\
        .option("port", 9999)\
        .load()

In [4]:
# .show/print will not work directily due to stream..
# linesDf.show() # worn't work
linesDf.printSchema() # works

root
 |-- value: string (nullable = true)



In [5]:
# split the lines into words, then convert the words into individual row using a function called explode
# explode will convert columns/array elements into spark record
import pyspark.sql.functions as F
# linesDf.value is a column
# split convert to list of words [welcome, to, spark]
# convert list of words into individual word/record
# explode, will convert elements into record
#wordsDf = linesDf.select(F.split(linesDf.value," "))
# after explode the output would be, column name is shown as col
#        welcome
#         to
#         spark
# wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")) )

wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word") )
#        welcome
#         to
#         spark
# now the same result with col name word

# generate running word count from stream
# "word" is a column name
wordCountsDf = wordsDf.groupBy("word").count()

In [7]:
# to print the data on console..
# read the data send by nc command from linux terminal, print it on Jupyter console
echoOnconsole = wordCountsDf\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [ ]:
echoOnconsole.awaitTermination()

# later you can terminal the jupyter